# Imports

In [ ]:
import numpy as np
from prettytable import PrettyTable
import matplotlib.pyplot as plt
from matplotlib.pyplot import subplots
import xraydb
# can install xraydb and pretttytable with pip

In [ ]:
# #iPython magic to make interactive plots work
# %matplotlib widget

# Sample Detector Distance Calculators

### Given a maximum q-value how far away can I put my detector?

In [ ]:
### User Inputs ###
length_y = 79.65 # (mm) total detector height (including detector gaps)
length_x = 77.1 # (mm) total detector width (including detector gaps)
energy = 8040 # (eV) Energy of X-rays
max_q = 0.8 # maximum q you would like to reach
bs_radius = 0.1 # (mm) radius of beamstop or minimum radius from beam center to acheive good data

### Calculations ###
wavelength = 12400/energy # in Å
max_theta = 2*(np.arcsin(max_q*wavelength/(4*np.pi)))

def calc_sdd(edge_d, corner_d, energy, max_theta):
    sdd_edge = edge_d/np.tan(max_theta)
    sdd_corner = corner_d/np.tan(max_theta)
    return sdd_edge, sdd_corner

def calc_min_q(bs_radius, energy, sdd):
    wavelength = 12400/energy # in Å
    min_q = 4*np.pi*np.sin(np.arctan(bs_radius/sdd)/2)/wavelength
    return min_q

# beam position = center
min_length = min(length_x/2, length_y/2)
edge_d = min_length
corner_d = ((length_y/2)**2 + (length_x/2)**2)**0.5
center_sdd_e, center_sdd_c = calc_sdd(edge_d, corner_d, energy, max_theta)
# beam position = bottom
min_length = min(length_x/2, length_y)
edge_d = min_length
corner_d = ((length_y)**2 + (length_x/2)**2)**0.5
bottom_sdd_e, bottom_sdd_c = calc_sdd(edge_d, corner_d, energy, max_theta)
#beam position = side
min_length = min(length_x, length_y/2)
edge_d = min_length
corner_d = ((length_y/2)**2 + (length_x)**2)**0.5
side_sdd_e, side_sdd_c = calc_sdd(edge_d, corner_d, energy, max_theta)
# beam position = corner
min_length = min(length_x, length_y)
edge_d = min_length
corner_d = ((length_y)**2 + (length_x)**2)**0.5
corner_sdd_e, corner_sdd_c = calc_sdd(edge_d, corner_d, energy, max_theta)

# plot table
table = PrettyTable()
table.add_column('', ['SDD (max ring)', 'SDD (corner)'])
table.add_column('Center Beam', [center_sdd_e, center_sdd_c])
table.add_column('Bottom Beam', [bottom_sdd_e, bottom_sdd_c])
table.add_column('Side Beam', [side_sdd_e, side_sdd_c])
table.add_column('Corner Beam', [corner_sdd_e, corner_sdd_c])
table.float_format = '.0'
print(f'Table showing SDDs (in mm) to put Q={max_q:.2f} Å-1 at the edge/corner of the detector')
print(table)

### Given an SDD and beam position what is the max q I will be able to detect?

In [ ]:
### User Inputs ###
length_y = 155.2 # (mm) total detector height (including detector gaps)
length_x = 162.5 # (mm) total detector width (including detector gaps)
energy = 12000 # (eV) Energy of X-rays
sdd = 10000 # (mm) sample detector distance
bs_radius = 4 # (mm) radius of beamstop or minimum radius from beam center to acheive good data


### Calculations ###
def calc_max_q(edge_d, corner_d, energy, sdd):
    wavelength = 12400/energy # in Å
    max_q_edge = 4*np.pi*np.sin(np.arctan(edge_d/sdd)/2)/wavelength
    max_q_corner = 4*np.pi*np.sin(np.arctan(corner_d/sdd)/2)/wavelength
    return max_q_edge, max_q_corner

def calc_min_q(bs_radius, energy, sdd):
    wavelength = 12400/energy # in Å
    min_q = 4*np.pi*np.sin(np.arctan(bs_radius/sdd)/2)/wavelength
    return min_q

# beam position = center
min_length = min(length_x/2, length_y/2)
edge_d = min_length
corner_d = ((length_y/2)**2 + (length_x/2)**2)**0.5
center_q_e, center_q_c = calc_max_q(edge_d, corner_d, energy, sdd)
# beam position = bottom
min_length = min(length_x/2, length_y)
edge_d = min_length
corner_d = ((length_y)**2 + (length_x/2)**2)**0.5
bottom_q_e, bottom_q_c = calc_max_q(edge_d, corner_d, energy, sdd)
#beam position = side
min_length = min(length_x, length_y/2)
edge_d = min_length
corner_d = ((length_y/2)**2 + (length_x)**2)**0.5
side_q_e, side_q_c = calc_max_q(edge_d, corner_d, energy, sdd)
# beam position = corner
min_length = min(length_x, length_y)
edge_d = min_length
corner_d = ((length_y)**2 + (length_x)**2)**0.5
corner_q_e, corner_q_c = calc_max_q(edge_d, corner_d, energy, sdd)
# minimum q
min_q = calc_min_q(bs_radius, energy, sdd)

# Convert min_q to scientific notation
min_q_sci = f"{min_q:.1e}"

# plot table
table = PrettyTable()
table.add_column('', ['Qmax (max ring)', 'Qmax (corner)', 'Qmin'])
table.add_column('Center Beam', [f"{center_q_e:.2f}", f"{center_q_c:.2f}", min_q_sci])
table.add_column('Bottom Beam', [f"{bottom_q_e:.2f}", f"{bottom_q_c:.2f}", min_q_sci])
table.add_column('Side Beam', [f"{side_q_e:.2f}", f"{side_q_c:.2f}", min_q_sci])
table.add_column('Corner Beam', [f"{corner_q_e:.2f}", f"{corner_q_c:.2f}", min_q_sci])
print(f'Table showing extreme Q values (Å-1) given an SDD of {sdd:.0f} mm')
print(table)

# Critical Angle Calculator

### Create Materials

In [ ]:
# Add your materials here. only needs to be ran once
# args: name, chemical formula, density (g/cc), categories (user defined)
# xraydb.add_material('SiN', 'Si3N4', 3.17, categories=['Ceramic'])
# xraydb.add_material('SiN', 'Si3N4', 3.17, categories=['Ceramic'])
# xraydb.add_material('lithium_iron_phosphate', 'LiFePO4', 3.60, categories=['Ceramic'])
# xraydb.add_material('PM6', 'C68H76F2O2S8', 1.2, categories=['Polymer'])
# xraydb.get_material('PM7')
# xraydb.get_materials()

### Calculate critical angles

In [ ]:
### User Inputs ###
energies = np.linspace(10000,13000,10) #energies of interest in eV
materials = ['PM7','PM6','Si', 'Li']
num_mats = len(materials)
### Calculations ###

def calc_crit_angle(delta):
    crit_rad = np.sqrt(2*delta)
    crit_deg = np.rad2deg(crit_rad)
    return crit_deg

all_crit_angles = []
for material in materials:
    mat_crit_angles = []
    for energy in energies:
        delta = xraydb.xray_delta_beta(xraydb.get_material(material)[0], xraydb.get_material(material)[1], energy)[0]
        crit_angle = calc_crit_angle(delta)
        mat_crit_angles.append(crit_angle)
    all_crit_angles.append(mat_crit_angles)
all_crit_angles = np.asarray(all_crit_angles)
np.shape(all_crit_angles)
#plot table
table = PrettyTable()
table.add_column('Energies (eV)', energies)
for i in range(num_mats):
    table.add_column(materials[i], all_crit_angles[i,:])
    table.float_format = '.4'
print(f'Critical Angles in degrees')
print(table)
# plt.figure(figsize=(10, 6))
# for i in range(num_mats):
#     plt.plot(energies, all_crit_angles[i,:], label=materials[i])
# plt.xlabel('Energy (eV)')
# plt.ylabel('Critical Angle (degs)')
# plt.title('Critical Angles in degrees')
# plt.legend()
# plt.grid(True)

# X-ray edges lookup

### Get all X-ray Edges for a given element

In [ ]:
element = 'Zn'
xray_edges = xraydb.xray_edges(element)
table = PrettyTable()
table.field_names = ["Edge", "Energy (eV)"]
# Adding rows to the table
for edge, data in xray_edges.items():
    table.add_row([edge, data[0]])
table.float_format = '.0'
print(f'X-ray Edges for {element}')
print(table)

# Single Material to multi-layer Attenuation and Transmission Calculator
#### This calculator will take your given material chemistry along with the density to calculate attenuation length and transmission

### Create Materials

In [ ]:
# Add your materials here. only needs to be ran once
# args: name, chemical formula, density (g/cc), categories (user defined)
# xraydb.add_material('SiN', 'Si3N4', 3.17, categories=['Ceramic'])
# xraydb.add_material('SiN', 'Si3N4', 3.17, categories=['Ceramic'])
# xraydb.add_material('lithium_iron_phosphate', 'LiFePO4', 3.60, categories=['Ceramic'])
# xraydb.add_material('PM6', 'C68H76F2O2S8', 1.2, categories=['Polymer'])
# xraydb.get_material('Fe')

### Calculate Material(s) Attenuation Lengths

In [ ]:
### User Inputs ###
energies = np.linspace(2400,2550,10) #energies of interest in eV
materials = ['PM7','PM6','Chlorobenzene','toluene','SiN']
num_layers = len(materials)
### Calculations ###
#calculate mu (cm-1)
material_mus = []
for i in range(num_layers):
    mu_val = xraydb.material_mu(materials[i], energies)
    material_mus.append(mu_val)
material_mus = np.asarray(material_mus)

#calculate attenuation length (mm)
material_atts = 10/material_mus

#plot table
table = PrettyTable()
table.add_column('Energies (eV)', energies)
for i in range(num_layers):
    table.add_column(materials[i], material_atts[i,:])
    table.float_format = '.4'
print(f'Attenuation lengths in mm')
print(table)
plt.figure(figsize=(10, 6))
for i in range(num_layers):
    plt.plot(energies, material_atts[i,:], label=materials[i])
plt.xlabel('Energy (eV)')
plt.ylabel('Attenuation Length (mm)')
plt.title('Attenuation Length vs Energy')
plt.legend()
plt.grid(True)

### Calculate Material(s) X-ray Transmission

In [ ]:
### User Inputs ###
num_layers = 3
energies = np.linspace(2400,2550,10) #energies of interest in eV
materials = ['PM7', 'toluene', 'SiN'] 
thicknesses = [0.00025, 0.025, 0.0002] #thicknesses in mm

### Calculations ###
#calculate mu (cm-1)
material_mus = []
for i in range(num_layers):
    mu_val = xraydb.material_mu(materials[i], energies)
    material_mus.append(mu_val)
material_mus = np.asarray(material_mus)

material_trans = []
total_trans = np.ones(np.shape(material_mus[0,:]))
for i in range(num_layers):
    trans_val = np.exp(thicknesses[i] * -0.1*material_mus[i,:])
    total_trans = total_trans*trans_val
    material_trans.append(trans_val)
material_trans = np.asarray(material_trans)

col_names = []
for i in range(num_layers):
    col_name = materials[i]+' ('+str(thicknesses[i])+' mm)'
    col_names.append(col_name)

#plot table
table = PrettyTable()
table.add_column('Energies (eV)', energies)
for i in range(num_layers):
    table.add_column(col_names[i], 100*material_trans[i,:])
table.add_column('Total Transmission', 100*total_trans)
table.float_format = '.0'
print(f'Transmission values in %')
print(table)

# Solution Attenuation and Transmission Calculator
#### This calculator will take your given solvent and solute chemistry along with the mass/volume of solute added to calculate attenuation and transmission
#### *Note: this calculator does not take into account excluded volume of the solute thereofore it will overestimate the absorption for significant excluded volume*

### Create Materials

In [ ]:
# Add your materials here. only needs to be ran once
# args: name, chemical formula, density (g/cc), categories (user defined)
# xraydb.add_material('leadsulfide', 'PbS6', 7.5, categories=['solvent', 'organic'])
# xraydb.add_material('chloronaphthalene', 'C10H7Cl', 1.2, categories=['solvent', 'organic'])
# xraydb.add_material('PM7', 'C68H76Cl2O2S8', 1.2, categories=['polymer', 'organic'])

### Calculate Solution Attenuation Length

In [ ]:
### User Inputs ###
energies = np.linspace(10000,13000,4) #energies of interest in eV
solvent = 'toluene'
solute = 'PM7'
solute_density = 0.01 #mass of solute in solution (g/ml)
cap_transmission = 0.95

### Calculations ###
#calculate mu (cm-1)
solvent_mus = xraydb.material_mu(solvent, energies)
solute_mus = xraydb.material_mu(solute, energies, density = solute_density)
# solute_mus = xraydb.material_mu(solute, energies)
total_mus = solvent_mus + solute_mus

#calculate attenuation length (mm)
solvent_atts = 10/solvent_mus
solute_atts = 10/solute_mus
total_atts = 10/total_mus

#plot table
table = PrettyTable()
table.add_column('Energies (eV)', energies)
table.add_column('Solvent Att. Length (mm)', solvent_atts)
table.add_column('Solute Att. Length (mm)', solute_atts)
table.add_column('Total Att. Length (mm)', total_atts)
table.float_format = '.2'
print(f'Attenuation lengths for {solute} solution at {solute_density:.2f} grams per ml of '
      f'{solvent}')
print(table)

### Calculate Solution Transmission

In [ ]:
### User Inputs ###
energies = np.linspace(8000,12000,5) #energies of interest in eV
solvent = 'toluene'
solute = 'PM7'
solute_density = 0.01 #mass of solute in solution (g/ml)
cap_diameter = 1.0 #path length in mm

### Calculations ###
#calculate mu (cm-1)
solvent_mus = xraydb.material_mu(solvent, energies)
solute_mus = xraydb.material_mu(solute, energies, density = solute_density)
total_mus = solvent_mus + solute_mus

#calculate transmission
solvent_trans = np.exp(cap_diameter * -0.1*solvent_mus)
solute_trans = np.exp(cap_diameter * -0.1*solute_mus)
total_trans = np.exp(cap_diameter * -0.1*total_mus)
table = PrettyTable()

#plot table
table.add_column('Energies (eV)', energies)
table.add_column('Solvent Transmission (%)', solvent_trans*100)
table.add_column('Solute Transmission (%)', solute_trans*100)
table.add_column('Total Transmission (%)', total_trans*100)
table.float_format = '.2'
print(f'Transmissions for {solute} solution at {solute_density:.2f} grams per ml of '
      f'{solvent} in a capillary diameter of {cap_diameter:.2f} mm')
print(table)